In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5 -O dogs_cats_model.h5

In [3]:
model = keras.models.load_model('dogs_cats_model.h5')

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [5]:
url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
image_size = (150, 150)

img = download_image(url)
img = prepare_image(img, target_size=image_size)

In [6]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X

array([[[[ 0.41176474,  0.54509807,  0.49803925],
         [ 0.38823533,  0.52156866,  0.45882356],
         [ 0.26274514,  0.39607847,  0.33333337],
         ...,
         [ 0.5294118 ,  0.7019608 ,  0.6784314 ],
         [ 0.36470592,  0.4901961 ,  0.4901961 ],
         [ 0.34901965,  0.5372549 ,  0.5529412 ]],

        [[ 0.05098045,  0.18431377,  0.13725495],
         [ 0.47450984,  0.60784316,  0.56078434],
         [ 0.21568632,  0.34901965,  0.28627455],
         ...,
         [ 0.58431375,  0.73333335,  0.7176471 ],
         [ 0.26274514,  0.4039216 ,  0.27843142],
         [ 0.5764706 ,  0.75686276,  0.70980394]],

        [[ 0.37254906,  0.49803925,  0.47450984],
         [ 0.23921573,  0.37254906,  0.32549024],
         [ 0.2941177 ,  0.427451  ,  0.36470592],
         ...,
         [ 0.20000005,  0.3176471 ,  0.2941177 ],
         [ 0.254902  ,  0.38823533,  0.30980396],
         [ 0.41176474,  0.5921569 ,  0.54509807]],

        ...,

        [[ 0.16078436,  0.23921573,  0

In [7]:
X.shape

(1, 150, 150, 3)

In [8]:
X[0][0][0][0]

0.41176474

In [9]:
pred = model.predict(X)

In [10]:
classes = [
    'dogs',
    'cats'
]

dict(zip(classes, pred[0]))

{'dogs': 0.9703119}

In [11]:
## Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/wk/yt_rdhdx7mqbrqjqngjkkqx00000gn/T/tmpwdhh98z9/assets


In [13]:
#!ls -lh

In [26]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
ERROR: Could not find a version that satisfies the requirement tflite_runtime (from versions: none)
ERROR: No matching distribution found for tflite_runtime


In [10]:
!pip install tflite_runtime-2.5.0-cp38-cp38-macosx_10_13_x86_64.whl

Processing ./tflite_runtime-2.5.0-cp38-cp38-macosx_10_13_x86_64.whl
tflite-runtime is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [12]:
import tensorflow.lite as tflite
#import tflite_runtime.interpreter as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='dogs_cats_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)

In [ ]:
classes = [
    'dogs',
    'cats'
]

dict(zip(classes, pred[0]))